# Scraping and Sentiment Analyzer

## Imports

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [3]:
#!pip install -q news-please pygooglenews cchardet transformers torch vaderSentiment xlsxwriter gspread gspread-pandas oauth2client
!pip install -q news-please 
!pip install -q Googlenews 
!pip install -q cchardet 
!pip install -q transformers 
!pip install -q torch
!pip install -q vaderSentiment 
!pip install -q xlsxwriter 
!pip install -q gspread 
!pip install -q gspread-pandas 
!pip install -q oauth2client
!pip -q install --upgrade oauth2client

In [4]:
import pandas
import numpy
from IPython.display import clear_output
from datetime import datetime, date, timedelta
from time import mktime
import math

import GoogleNews
from newsplease import NewsPlease
import torch
import requests
from transformers import BertTokenizer, BertForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.tokenize import sent_tokenize
from  matplotlib.colors import LinearSegmentedColormap

import gspread
import gspread_pandas

[nltk_data] Downloading package punkt to C:\Users\Advait
[nltk_data]     Lath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to C:\Users\Advait
[nltk_data]     Lath\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
#nltk.download()


## News Scraping

### News Fetcher

In [6]:

def get_news(google_news_code):
  gn = GoogleNews(lang = 'en', country = 'IN')
  company_news = gn.topic_headlines(google_news_code)
  entries = []
  rows = []
  # can change sources below
  sources = ['bloombergquint', 'business-standard', 'cnbctv18',
             'economictimes', 'financialexpress', 'indiainfoline',
             'livemint', 'moneycontrol', 'ndtv',
             'newindianexpress', 'news18', 'thehindubusinessline',
             'timesofindia', 'zeebiz']
  # change date range below 
  date_filter = pandas.to_datetime((date.today() - timedelta(days=2)),
                                   format='%Y-%m-%d')
 
  for entry in company_news['entries']:
    upload_date = (datetime.fromtimestamp(mktime(entry['published_parsed']))).strftime('%Y-%m-%d')
    upload_date = pandas.to_datetime(upload_date,format='%Y-%m-%d')
    if upload_date>=date_filter:
      #make the below "not in" and this will solve the paid article issue
      if any(source in entry['source']['href'] for source in sources):
        entries.append(entry)

  for entry in entries:
    try:
      link = requests.get(entry['link'])
      time_of_upload = datetime.fromtimestamp(mktime(entry['published_parsed']))
      link = link.url
      title = entry['title']
      article = NewsPlease.from_url(link, timeout=10)
      row=[time_of_upload,link,title,article.description,article.maintext]
      rows.append(row)
    except:
      pass

  df = pandas.DataFrame(rows, columns=['Date','URL','Title','Description','Text'])
  df[df.columns[1:]] = df[df.columns[1:]].astype(str)
  return df

### News cleaning

In [7]:
def cleaner(df):
    data = df[['Title','Description','Text']]
    texts = data['Text'].tolist()
    
    for text in texts:
        if (len(data[data['Text']==text])>=2):
            data['Text'].replace(to_replace=text,value="None",inplace=True)
    df[['Title','Description','Text']] = data
    for column in df.columns:
        df[column] = df[column].astype(str)
    df['Date'] = pandas.to_datetime(df['Date'], format='%Y-%m-%d', errors='ignore')
    
    df['Text'] = df['Text'].str.lower()
    # for impurity in impurities:
    #      df['Text'] = df['Text'].str.replace(impurity,' ',regex=True)
    
    df = df.reset_index()
    return df.sort_values(by='Date', ascending=False)

### News driver

In [8]:

companies = pandas.read_excel('D:\\Downloads\\Sentiment analysis\\backend\\Sentiment Analysis\\Output CSVs\\BSE100.xlsx', index_col=0)
#companies = pandas.read_csv('D:\\Downloads\\Sentiment analysis\\backend\\Sentiment Analysis\\GNews Companies URLs\\BSE100.csv')
frames = [companies]
for num in range(0,len(companies[:])):
    clear_output(wait=True)
    try:
        news_df = get_news(companies['Code'][num])
        news_df.insert(0,'GNews',('https://news.google.com/topics/'+companies['Code'][num]+'?hl=en-IN&gl=IN&ceid=IN%3Aen'))
        news_df.insert(0,'Name',companies['Name'][num])
        print((num+1),': ',news_df['Name'][0])
        print('Rows: ',len(news_df['Name']))
        if len(news_df)>=1:
            frames.append(news_df)
    except:
        pass
news= pandas.concat(frames)


In [9]:
news

,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
index,,,,,,,,,,,,
54,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,22/02/22 3:06,https://www.business-standard.com/article/mark...,"Axis Bank, India Cement: Mehul Kothari bets on...","Read more about Axis Bank, India Cement: Mehul...",buy | target rs 840 | stop loss rs 750 a coupl...,40,positive,-17,negative,NaN
53,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,22/02/22 3:53,https://economictimes.indiatimes.com/markets/s...,Strong upside likely in these 5 stocks after o...,A total of 47 analysts have a median target of...,none,75,neutral,75,positive,NaN
55,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,22/02/22 8:55,https://www.livemint.com/market/stock-market-n...,Nifty below 17000 offer a good risk-reward tra...,One should use this correction in Indian stock...,indian stock markets dropped sharply amid a se...,-77,negative,82,positive,NaN
56,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,23/02/22 7:50,https://www.zeebiz.com/market-news/news-kotak-...,Kotak Mutual Fund AUM sees 4500 cr jump in Jan...,Kotak Mutual Fund saw it assets under manageme...,none,80,neutral,80,positive,NaN
670,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,2021-01-09 07:00:00,https://www.cnbctv18.com/market/stocks/axis-ba...,Axis Bank hits new 52-week high; partners with...,Axis Bank share price hit a 52-week high on BS...,shares of axis bank hit a 52 week high during ...,82,positive,65,positive,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3008,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,27/01/22 8:00,https://www.business-standard.com/article/news...,"Wipro, Cipla, United Spirits to be watched - B...","Read more about Wipro, Cipla, United Spirits t...",wipro the it major completed the acquisition o...,61,positive,82,positive,NaN
3007,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,28/01/22 8:00,https://www.cnbctv18.com/business/diageo-owned...,Diageo-owned United Spirits prepping platform ...,The Diageo-owned company will use its In.theba...,"diageo owned united spirits ltd , which makes ...",55,neutral,43,neutral,NaN
3006,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,30/01/22 8:00,https://www.bloombergquint.com/research-report...,United Spirits Q3 Review - Performance Below O...,United Spirits’ Q3FY22 performance came in bel...,united spirits ltd.’s q3 fy22 performance came...,-78,negative,-77,negative,NaN


In [10]:
#Cleaning the news
news = cleaner(news)

c:\Users\Advait Lath\anaconda3\lib\site-packages\pandas\core\generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [11]:
news.drop_duplicates()

,index,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
1002,1031,Sun Pharmaceutical Industries Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/mark...,"Sun Pharma, TCS, UltraTech: 5 Sensex stocks th...","Read more about Sun Pharma, TCS, UltraTech: 5 ...",none,77,neutral,77,positive,nan
33,641,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy Axis Bank, target price Rs 690: Kotak Secu...","Axis Bank Ltd., incorporated in the year 1993,...",agencies promoters held 11.64 per cent stake i...,67,positive,29,neutral,nan
825,21,Reliance Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.ndtv.com/business/reliance-industr...,Reliance Arm Acquires 100% Stake In UK-Based F...,Reliance New Energy Solar Limited - a wholly-o...,reliance s renewable energy arm will invest an...,84,positive,83,positive,nan
573,542,ITC Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/industry/...,ITC to launch plant-based meat products - Econ...,The company is the first mainstream consumer c...,none,73,neutral,73,neutral,nan
1165,3011,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/news...,United Spirits receives revision in credit rat...,Read more about United Spirits receives revisi...,none,62,neutral,62,positive,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,712,Larsen & Toubro Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,13/01/22 8:00,https://www.business-standard.com/article/mark...,Larsen & Toubro hits new all-time high on winn...,Read more about Larsen & Toubro hits new all-t...,shares of larsen & toubro hit a new high of rs...,77,positive,73,positive,nan
1167,3010,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy United Spirits, target price Rs 965: IIFL ...","United Spirits Ltd., incorporated in the year ...",getty images promoters held 56.73 per cent sta...,66,positive,46,neutral,nan
329,106,HDFC Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://economictimes.indiatimes.com/wealth/bo...,SBI vs ICICI Bank vs HDFC Bank vs Bank of Baro...,Those looking to take a car loan must shop aro...,"for many owning a car is no longer a luxury, r...",21,neutral,15,neutral,nan
881,1674,SBI Cards And Payment Services Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://www.livemint.com/money/personal-financ...,"Credit card late fee: What HDFC Bank, SBI Card...",ICICI Bank has revised charges for credit card...,none,29,neutral,29,neutral,nan


## Sentiment Analyser

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') #older version
#tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
#tokenizer = BertTokenizer.from_pretrained("ProsusAI/finbert")
model = BertForSequenceClassification.from_pretrained("D:\\Downloads\\Sentiment analysis\\backend\\Sentiment Analysis\\Model files\\pytorch_model.bin",
                                                      config="D:\\Downloads\\Sentiment analysis\\backend\\Sentiment Analysis\\Model files\\config.json",
                                                      num_labels=3)
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

#model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels= 3)                                                     
label_list=['positive','negative','neutral']
analyser = SentimentIntensityAnalyzer()

Some weights of the model checkpoint at D:\Downloads\Sentiment analysis\backend\Sentiment Analysis\Model files\pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForS

##Running finBERT

In [13]:
# def finbert_model(input_df, article_part="Text"):
#   ml_polarity = []
#   ml_sent = []
#   for idx in input_df[article_part]:
#    inputs = tokenizer(idx, return_tensors="pt", padding=True, truncation= True)
#    outputs = model(**inputs)[0]
#    vader = analyser.polarity_scores(idx)
#    labels = {0:'neutral', 1:'positive',2:'negative'}
#    if (idx==str(numpy.nan)):
#     ml_sent.append('neutral')
#     ml_polarity.append(0)
#    else:
#      ml_sent.append(labels[torch.argmax(outputs[0])])
#      ml_polarity.append(vader['compound'])
#     # print(sent, '----', labels[numpy.argmax(outputs.detach().numpy()[idx])])
#     # vader['compound']
#   input_df[article_part+'Polarity'] = ml_polarity
#   input_df[article_part+'Sentiment'] = ml_sent
#   return input_df  
    

In [14]:
# def finbert_model(input_df,article_part="Text"):
#     ml_polarity = []
#     ml_sent = []
#     for i in input_df[article_part]:
#      inputs = tokenizer(i, return_tensors="pt", truncation=True)
#      outputs = model(**inputs)
#      prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
#      prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
#      polarity_score = prob_0-prob_1
#     if (i==str(numpy.nan)):
#       ml_sent.append('neutral')
#       ml_polarity.append(0)
#     else:
#       ml_sent.append(label_list[torch.argmax(outputs[0])])
#       ml_polarity.append(polarity_score)  
#     input_df[article_part+'Polarity'] = ml_polarity
#     input_df[article_part+'Sentiment'] = ml_sent
#     return input_df  

In [15]:
# f =lambda Text: analyser.polarity_scores(Text)['compound']
# news['TextPolarity']=news['TextPolarity'].apply(f)
# news['DescriptionPolarity']=news['DescriptionPolarity   '].apply(f)

In [16]:
def finbert_model(input_df, article_part="Text"):
  ml_polarity = []
  ml_sent = []
  vader_polarity = []
  # doing the analysis for the text column
  for i in input_df[article_part]:
    inputs = tokenizer(i, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
    prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
    polarity_score = prob_0-prob_1
    vader = analyser.polarity_scores(i)
    if (i==str(numpy.nan)):
      ml_sent.append('neutral')
      ml_polarity.append(0)
      vader_polarity.append(0)
    else:
      ml_sent.append(label_list[torch.argmax(outputs[0])])
      #ml_sent.append(label_list[torch.argmax(outputs.logits)])
      ml_polarity.append(polarity_score)
      vader_polarity.append(vader['compound'])
  input_df[article_part+'Polarity'] = ml_polarity
  input_df[article_part+'Sentiment'] = ml_sent
  return input_df

In [17]:
# def finbert_model(input_df, article_part="Text"):
#   ml_polarity = []
#   ml_sent = []
#   vader_polarity = []
#   # doing the analysis for the text column
#   for i in input_df[article_part]:
#     inputs = tokenizer(i, return_tensors="pt", truncation=True)
#     outputs = model(**inputs)
#     prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
#     prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
#     polarity_score = prob_0-prob_1
#     ml_sentiment = label_list[torch.argmax(outputs[0])]
#     #vader = analyzer.polarity_scores(i)
#     vader = analyser.polarity_scores(i)
#     if (i==str(numpy.nan)):
#       ml_sent.append('neutral')
#       ml_polarity.append(0)
#       vader_polarity.append(0)
#     else:
#       ml_sent.append(label_list[torch.argmax(outputs[0])])
#       ml_polarity.append(polarity_score)
#       vader_polarity.append(vader['compound'])
#   input_df[article_part+'Polarity'] = ml_polarity
#   input_df[article_part+'Sentiment'] = ml_sent
#   return input_df

In [18]:
#news['compound']  = news['scores'].apply(lambda score_dict: score_dict['compound'])
#news['Text Sentiment'] = news['Description'].apply(lambda Description: analyser.polarity_scores(Description))
#will go up
# cs = []
# for row in range(len(final_news)):
#     cs.append(analyzer.polarity_scores(final_news['title'].iloc[row])['compound'])

In [19]:
#Running model on text
news = finbert_model(news, article_part="Text")


In [20]:
#Running model on description 
news = finbert_model(news, article_part="Description")

In [21]:
# f =lambda Text: analyser.polarity_scores(Text)['compound']
# news['TextPolarity']=news['TextPolarity'].apply(f)
# news['DescriptionPolarity']=news['DescriptionPolarity   '].apply(f)

In [22]:
# news['TextPolarity']  = news['TextPolarity'].apply(lambda score_dict: score_dict['compound'])
#news['Text Sentiment'] = news['Description'].apply(lambda Description: analyser.polarity_scores(Description))

In [23]:
news

,index,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
1002,1031,Sun Pharmaceutical Industries Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/mark...,"Sun Pharma, TCS, UltraTech: 5 Sensex stocks th...","Read more about Sun Pharma, TCS, UltraTech: 5 ...",none,0.145790,neutral,0.162740,neutral,nan
33,641,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy Axis Bank, target price Rs 690: Kotak Secu...","Axis Bank Ltd., incorporated in the year 1993,...",agencies promoters held 11.64 per cent stake i...,0.190033,positive,0.159806,neutral,nan
825,21,Reliance Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.ndtv.com/business/reliance-industr...,Reliance Arm Acquires 100% Stake In UK-Based F...,Reliance New Energy Solar Limited - a wholly-o...,reliance s renewable energy arm will invest an...,0.145139,neutral,0.164506,neutral,nan
573,542,ITC Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/industry/...,ITC to launch plant-based meat products - Econ...,The company is the first mainstream consumer c...,none,0.145790,neutral,0.158656,neutral,nan
1165,3011,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/news...,United Spirits receives revision in credit rat...,Read more about United Spirits receives revisi...,none,0.145790,neutral,0.193032,neutral,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,712,Larsen & Toubro Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,13/01/22 8:00,https://www.business-standard.com/article/mark...,Larsen & Toubro hits new all-time high on winn...,Read more about Larsen & Toubro hits new all-t...,shares of larsen & toubro hit a new high of rs...,0.161708,neutral,0.157929,neutral,nan
1167,3010,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy United Spirits, target price Rs 965: IIFL ...","United Spirits Ltd., incorporated in the year ...",getty images promoters held 56.73 per cent sta...,0.164836,neutral,0.159331,neutral,nan
329,106,HDFC Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://economictimes.indiatimes.com/wealth/bo...,SBI vs ICICI Bank vs HDFC Bank vs Bank of Baro...,Those looking to take a car loan must shop aro...,"for many owning a car is no longer a luxury, r...",0.149713,neutral,0.153486,neutral,nan
881,1674,SBI Cards And Payment Services Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,13/01/22 8:00,https://www.livemint.com/money/personal-financ...,"Credit card late fee: What HDFC Bank, SBI Card...",ICICI Bank has revised charges for credit card...,none,0.145790,neutral,0.173257,neutral,nan


In [24]:
# def sentiment_score(input_df,article_part='Text'):
#     vader_polarity = []
#     for i in input_df[article_part]:
#         vader = analyser.polarity_scores(i)
#     if (i==str(numpy.nan)):
#         vader_polarity.append(0)
#     else:
#        vader_polarity.append(vader['compound']) 
#     input_df[article_part+'Polarity'] = vader_polarity
#     return input_df    


In [25]:
news.head(5)

,index,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
1002,1031,Sun Pharmaceutical Industries Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/mark...,"Sun Pharma, TCS, UltraTech: 5 Sensex stocks th...","Read more about Sun Pharma, TCS, UltraTech: 5 ...",none,0.145790,neutral,0.162740,neutral,nan
33,641,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy Axis Bank, target price Rs 690: Kotak Secu...","Axis Bank Ltd., incorporated in the year 1993,...",agencies promoters held 11.64 per cent stake i...,0.190033,positive,0.159806,neutral,nan
825,21,Reliance Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.ndtv.com/business/reliance-industr...,Reliance Arm Acquires 100% Stake In UK-Based F...,Reliance New Energy Solar Limited - a wholly-o...,reliance s renewable energy arm will invest an...,0.145139,neutral,0.164506,neutral,nan
573,542,ITC Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/industry/...,ITC to launch plant-based meat products - Econ...,The company is the first mainstream consumer c...,none,0.145790,neutral,0.158656,neutral,nan
1165,3011,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/news...,United Spirits receives revision in credit rat...,Read more about United Spirits receives revisi...,none,0.145790,neutral,0.193032,neutral,nan


In [26]:
def most_extreme(input_df):
    extreme_polarities = []
    extreme_sentences = []
    input_df.dropna()
    for text in input_df["Text"]:
        extreme_polarity = 0
        extreme_sentence = "None"

        if len(text)>10:
            sentences = sent_tokenize(text)
            
            for sentence in sentences:
                input = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
                outputs = model(**input)
                prob_0 = math.exp(outputs[0][0][0])/(1+math.exp(outputs[0][0][0]))
                prob_1 = math.exp(outputs[0][0][1])/(1+math.exp(outputs[0][0][1]))
                ml_sentiment = label_list[torch.argmax(outputs[0])]
                polarity_score = prob_0-prob_1
                if abs(polarity_score)>extreme_polarity:
                    extreme_polarity = polarity_score
                    extreme_sentence = sentence
        extreme_polarities.append(extreme_polarity)
        extreme_sentences.append(extreme_sentence)

    input_df["Extreme Polarity"] = extreme_polarities
    input_df["Extreme Sentence"] = extreme_sentences
    return input_df

In [27]:
#news['TextPolarity'] = news['TextPolarity'].apply(lambda vader: analyser.polarity_scores (text="Text"))

In [28]:
news.to_csv("Sample1.csv")

## Data Cleaning

In [29]:
# cols_selected = ['index']
# cols_drop = []
news.drop("index",inplace=True,axis=1)
# news.drop('index',inplace=True,axis=1)

In [30]:
news.head(10)

,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
1002,Sun Pharmaceutical Industries Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/mark...,"Sun Pharma, TCS, UltraTech: 5 Sensex stocks th...","Read more about Sun Pharma, TCS, UltraTech: 5 ...",none,0.145790,neutral,0.162740,neutral,nan
33,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/markets/s...,"Buy Axis Bank, target price Rs 690: Kotak Secu...","Axis Bank Ltd., incorporated in the year 1993,...",agencies promoters held 11.64 per cent stake i...,0.190033,positive,0.159806,neutral,nan
825,Reliance Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.ndtv.com/business/reliance-industr...,Reliance Arm Acquires 100% Stake In UK-Based F...,Reliance New Energy Solar Limited - a wholly-o...,reliance s renewable energy arm will invest an...,0.145139,neutral,0.164506,neutral,nan
573,ITC Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://economictimes.indiatimes.com/industry/...,ITC to launch plant-based meat products - Econ...,The company is the first mainstream consumer c...,none,0.145790,neutral,0.158656,neutral,nan
1165,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,31/12/21 8:00,https://www.business-standard.com/article/news...,United Spirits receives revision in credit rat...,Read more about United Spirits receives revisi...,none,0.145790,neutral,0.193032,neutral,nan
76,Bajaj Finserv Ltd,https://news.google.com/topics/CAAqKQgKIiNDQkF...,31/12/21 8:00,https://www.cnbctv18.com/photos/market/stocks/...,"Tata Motors, Bajaj Finserv, IEX and more: Top ...",Top stocks of 2021: The calendar year 2021 wil...,none,0.145790,neutral,0.158146,neutral,nan
407,Hindalco Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.cnbctv18.com/photos/market/stocks/...,"IDFC, Hindalco, Titan, CMS Info Systems and mo...",Indian shares ended the last trading session o...,1 / 6 hindalco industries | the stock was the ...,0.162133,neutral,0.170263,neutral,nan
1119,Tech Mahindra Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.livemint.com/companies/news/tech-m...,Tech Mahindra to acquire 100% stake in US-base...,Tech Mahindra will pay total consideration of ...,it major tech mahindra on friday announced tha...,0.153394,neutral,0.161895,neutral,nan
406,Hindalco Industries Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/12/21 8:00,https://www.cnbctv18.com/market/stocks/hindalc...,Hindalco shares jump 5%; why metal stock is to...,Hindalco share price: Shares of Hindalco Indus...,shares of hindalco industries gained after two...,0.155850,neutral,0.161802,neutral,nan
733,NTPC Ltd,https://news.google.com/topics/CAAqJggKIiBDQkF...,31/10/21 7:00,https://www.livemint.com/news/india/ntpc-state...,"NTPC, states procuring biomass pellets to cont...",NTPC Ltd plans to procure 2.5 million tonne of...,new delhi as part of the centre’ plan to conta...,0.156072,neutral,0.170720,neutral,nan


In [31]:
news.sort_values(by=['Name', 'Date'], inplace=True)

In [32]:
# news['TextPolarity'] = (news['TextPolarity']*100).round(2)
# news['DescriptionPolarity'] = (news['DescriptionPolarity']*100).round(2)

In [33]:
news['TextPolarity'] = (news['TextPolarity']*100).round(2)
news['DescriptionPolarity'] = (news['DescriptionPolarity']*100).round(2)
import tldextract
import pandas as pd
news['House'] = news['URL'].apply(lambda x: tldextract.extract(x)[1])
# news ['Date'] = pd.to_datetime(news['Date'],format="%d/%m/%Y", errors='coerce')
# news['TextPolarity'] = (news['TextPolarity']*100).round()
# news['DescriptionPolarity'] = (news['DescriptionPolarity']*100).round()

In [34]:
news

,Name,GNews,Date,URL,Title,Description,Text,TextPolarity,TextSentiment,DescriptionPolarity,DescriptionSentiment,House
50,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,14/02/22 6:01,https://www.livemint.com/industry/banking/axis...,Axis Bank set to buy Citigroup’s India retail ...,An agreement for the consumer unit may be anno...,axis bank ltd. is nearing a deal to buy citigr...,16.17,neutral,16.59,neutral,livemint
6,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,14/09/21 7:00,https://www.thehindubusinessline.com/money-and...,Axis Bank lists $600 million Sustainable $AT1 ...,"Listed on Singapore, London and India INX and ...",axis bank on thursday announced the listing of...,15.99,neutral,16.11,neutral,thehindubusinessline
25,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,14/12/21 8:00,https://economictimes.indiatimes.com/industry/...,Axis Bank board appoints Bain Capital's Ashish...,The private sector lender said that Kotecha's ...,axis bank on tuesday said its board has approv...,14.37,neutral,15.13,neutral,indiatimes
51,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,15/02/22 12:20,https://www.bloombergquint.com/research-report...,Manappuram Finance Q3 Review - Muted Performan...,Manappuram Finance Ltd.’s Q3 FY22 earnings per...,manappuram finance ltd.’s q3 fy22 earnings per...,14.48,neutral,14.80,neutral,bloombergquint
40,Axis Bank Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,17/01/22 8:00,https://www.livemint.com/industry/banking/axis...,Axis Bank closing in on Citi India's consumer ...,"Another Indian lender, Kotak Mahindra Bank is ...",axis bank has emerged as the frontrunner to bu...,16.04,neutral,17.67,neutral,livemint
...,...,...,...,...,...,...,...,...,...,...,...,...
1171,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,30/01/22 8:00,https://www.bloombergquint.com/research-report...,United Spirits Q3 Review - Performance Below O...,United Spirits’ Q3FY22 performance came in bel...,united spirits ltd.’s q3 fy22 performance came...,15.57,neutral,15.80,neutral,bloombergquint
1149,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,30/09/21 7:00,https://www.indiainfoline.com/article/news-top...,"NCLT, Bengaluru directs Pioneer Distilleries t...",Shareholders on September 30 will consider the...,none,14.58,neutral,16.30,neutral,indiainfoline
1150,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,30/09/21 7:00,https://www.business-standard.com/article/news...,United Spirits Ltd in demand - Business Standard,Read more about United Spirits Ltd in demand o...,"united spirits ltd is quoting at rs 852.4, up ...",15.20,neutral,16.74,neutral,business-standard
1161,United Spirits Ltd,https://news.google.com/topics/CAAqKAgKIiJDQkF...,30/11/21 8:00,https://www.livemint.com/market/stock-market-n...,₹8.86 to ₹886: Multibagger stock turns ₹1 lakh...,Multibagger stock: If an investor had invested...,multibagger stock united spirits shares are on...,15.68,neutral,16.22,neutral,livemint


In [35]:
news['TextPolarity'].where(news['Text']!="none", news['DescriptionPolarity'], inplace=True)

In [36]:
news.to_excel('sample.xlsx',)


## Outputting to Google Sheets via gspread

In [37]:
!pip install gspread-dataframe
%pip install gspread-dataframe
!pip3 install gspread
!pip3 install --upgrade google-api-python-client oauth2client 

Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.51.0
    Uninstalling google-api-python-client-2.51.0:
      Successfully uninstalled google-api-python-client-2.51.0


In [38]:
from oauth2client.service_account import ServiceAccountCredentials
from gspread_pandas import Spread, Client
from datetime import datetime as dt
import gspread_dataframe as gd

In [39]:
scope = ['https://www.googleapis.com/auth/spreadsheets', 
'https://www.googleapis.com/auth/drive.file',
'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('Sentiment analysis\\backend\\Sentiment analysis\\Output CSVs\\Google Sheet-GSpread API stuff\\credentials.json.json',
                                                         scope)
client = gspread.authorize(creds)


In [40]:
config = gspread_pandas.conf.get_config(conf_dir='Sentiment analysis\\backend\\Sentiment analysis\\Output CSVs\\Google Sheet-GSpread API stuff',
                               file_name='credentials.json.json')

In [41]:
s = gspread_pandas.spread.Spread("url here", config = config, sheet="nimf-baf")

In [42]:
s.sheets

[<Worksheet 'nimf-baf' id:0>]

In [43]:
s.url

'https://docs.google.com/spreadsheets/d/1wYA3OyT0biSr6_eqWShtHuLS9CvfHRzhZ_xPEIVQzPI'

In [44]:
news.shape
#wks = client.open_by_key('1wYA3OyT0biSr6_eqWShtHuLS9CvfHRzhZ_xPEIVQzPI')



(1174, 12)

In [45]:
start_cell = "A"+str(s.get_sheet_dims()[0]+1)

In [46]:
existing_data_from_gsheet = s.sheet_to_df(header_rows=1)

In [ ]:
appended_df = existing_data_from_gsheet.append(news)

In [ ]:
appended_df.sort_values(by=['Name', 'Date'], inplace=True)

In [ ]:

s.df_to_sheet(df=appended_df, sheet="nimf-baf", start="A1", replace=True,
              freeze_headers=True)
